In [ ]:
from IPython.core.display import HTML
with open('../style.css') as f:
    css = f.read()
HTML(css)

In [ ]:
%load_ext nb_mypy

# Iterative Deepening

In [ ]:
from typing import TypeVar, Callable

In [ ]:
State    = TypeVar('State')
NxtStFct = Callable[[State], set[State]] 

The function `search` takes three arguments to solve a *search problem*:
- `start` is the start state of the search problem,
- `goal` is the goal state, and
- `next_states` is a function with signature $\texttt{next_states}:Q \rightarrow 2^Q$, where $Q$ is the set of states.
  For every state $s \in Q$, $\texttt{next_states}(s)$ is the set of states that can be reached from $s$ in one step.
If successful, `search` returns a path from `start` to `goal` that is a solution of the search problem
$$ \langle Q, \texttt{next_states}, \texttt{start}, \texttt{goal} \rangle. $$

The procedure `search` tries to find a solution to the search problem by first trying to find a solution that has a length of $1$, then of length $2$, then of length $3$, etc. 
The search only stops when a solution is found.  

We need the following forward declaration for the type checker.

In [ ]:
def depth_limited_search(Path: list[State], goal: State, next_states: NxtStFct, limit: int) -> list[State] | None:
    return None 

In [ ]:
def search(start: State, goal: State, next_states: NxtStFct) -> list[State]:
    limit = 32
    while True:
        Path = depth_limited_search([start], goal, next_states, limit)
        if Path is not None:
            return Path
        limit += 1
        print(f'limit = {limit}')

The function `depth_limited_search` tries to find a solution to the search problem
$$ \langle Q, \texttt{next_states}, \texttt{start}, \texttt{goal} \rangle $$
that has a length of at most `limit`.  The algorithm used is *depth limited search*,
i.e. we are running depth first search with a limit on the length of the path.
* `Path` is a path that start from the node `start`.
* `goal` is the node that is searched for, i.e. the function tries to extend `Path` so that it end with `goal`.
* `next_states` computes the set of states that can be reached in on step from a given state.
* `limit` is the maximal length that the completed path is allowed to have.

In [ ]:
def depth_limited_search(Path: list[State], goal: State, next_states: NxtStFct, limit: int) -> list[State] | None:
    state = Path[-1]
    if state == goal:
        return Path
    if len(Path) == limit:
        return None
    for ns in next_states(state):
        if ns not in Path:
            Result = depth_limited_search(Path + [ns], goal, next_states, limit)
            if Result:
                return Result
    return None

# Solving the Sliding Puzzle

In [ ]:
%run 03-Sliding-Puzzle.ipynb

In [ ]:
%load_ext memory_profiler

For the $3 \times 3$ sliding puzzle, computing the solution with iterative deepening takes about 3 seconds if I start with a depth limit of 32, which is the length of the list `Path` of the optimal solution.  However, if I start with a depth limit of 1, it takes about 7 minutes on my Mac Studio from 2022 that is equipped with an Apple M1 Max chip.

In [ ]:
Path: tuple[tuple[int, ...], ...]

In [ ]:
%%time
%memit Path = search(start, goal, next_states)

In [ ]:
len(Path) 

In [ ]:
animation(Path) # type: ignore